# Task 10.6: LLaMA Factory Exploration

**Module:** 10 - Large Language Model Fine-Tuning  
**Time:** 2 hours  
**Difficulty:** ⭐⭐☆☆☆

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Install and configure LLaMA Factory
- [ ] Use the web UI to configure fine-tuning
- [ ] Monitor training progress through the interface
- [ ] Compare GUI vs script-based workflows
- [ ] Understand when to use each approach

---

## What is LLaMA Factory?

**LLaMA Factory** is an open-source project that provides a unified, easy-to-use interface for fine-tuning LLMs. It supports:

- Multiple training methods (SFT, DPO, RLHF, PPO)
- Many model families (Llama, Mistral, Qwen, etc.)
- LoRA, QLoRA, and full fine-tuning
- Web UI and command-line interfaces
- Built-in evaluation and export

**Why use it?**
- Visual configuration with no coding
- Preset configurations for common tasks
- Real-time training monitoring
- Great for beginners and quick experiments

---

## ELI5: Why Use a GUI for Fine-Tuning?

> **Imagine learning to drive:**
>
> **Script-based training** is like learning to drive a manual transmission. You have full control over every gear shift, but you need to understand clutch timing, engine RPM, and coordination. Powerful, but steep learning curve.
>
> **LLaMA Factory** is like driving an automatic. The car handles the complex mechanics while you focus on where you want to go. You might sacrifice some control, but you get there faster and with less stress.
>
> **When to use which:**
> - **GUI (LLaMA Factory)**: Quick experiments, learning, prototyping
> - **Scripts**: Production pipelines, custom requirements, maximum control

---

## Part 1: Installation

Let's install LLaMA Factory on your DGX Spark.

In [ ]:
# Installation commands
# Run these in your terminal, not in the notebook

installation_commands = """
# Step 1: Clone the repository
git clone https://github.com/hiyouga/LLaMA-Factory.git
cd LLaMA-Factory

# Step 2: Create conda environment (recommended)
conda create -n llama_factory python=3.10
conda activate llama_factory

# Step 3: Install dependencies
pip install -e ".[torch,metrics]"

# Step 4: Verify installation
python -c "import llamafactory; print('LLaMA Factory installed successfully!')"

# Step 5: Launch the web UI
llamafactory-cli webui
"""

print("LLaMA Factory Installation Commands:")
print("="*50)
print(installation_commands)

In [ ]:
# Alternative: Docker installation (recommended for DGX Spark)
docker_commands = """
# Option A: Using Docker (cleaner, isolated)

# Pull the image
docker pull hiyouga/llama-factory:latest

# Run with GPU support
docker run --gpus all -it --rm \\
    -v $HOME/workspace:/workspace \\
    -v $HOME/.cache/huggingface:/root/.cache/huggingface \\
    -p 7860:7860 \\
    --ipc=host \\
    hiyouga/llama-factory:latest \\
    llamafactory-cli webui --host 0.0.0.0

# Notes:
# --gpus all      : Required for GPU access
# --ipc=host      : Required for DataLoader multiprocessing (prevents shared memory errors)
# -p 7860:7860    : Web UI port mapping

# Access the web UI at: http://localhost:7860

# ============================================================
# DGX SPARK ARM64 COMPATIBILITY NOTE
# ============================================================
# DGX Spark uses ARM64 (aarch64) architecture. If you encounter 
# architecture-related issues with the LLaMA Factory Docker image:
#
# Option 1: Use the NGC PyTorch container + pip install
#   docker run --gpus all -it --rm \\
#       -v $HOME/workspace:/workspace \\
#       -v $HOME/.cache/huggingface:/root/.cache/huggingface \\
#       -p 7860:7860 --ipc=host \\
#       nvcr.io/nvidia/pytorch:25.03-py3 bash
#   
#   # Inside container:
#   pip install llamafactory[torch,metrics]
#   llamafactory-cli webui --host 0.0.0.0
#
# Option 2: Build from source with ARM64 base image
#   git clone https://github.com/hiyouga/LLaMA-Factory.git
#   cd LLaMA-Factory
#   # Modify Dockerfile to use ARM64-compatible base image
#   docker build -t llama-factory-arm64 .
#
# The NGC container approach (Option 1) is recommended for DGX Spark
# as it includes ARM64-optimized PyTorch and CUDA libraries.
"""

print("Docker Installation (Recommended for DGX Spark):")
print("="*50)
print(docker_commands)

---

## Part 2: Web UI Overview

The LLaMA Factory web UI has several main sections.

In [ ]:
# Web UI Components
ui_components = """
LLAMA FACTORY WEB UI - MAIN COMPONENTS
======================================

1. MODEL SELECTION
   ├── Model Name: Choose from supported models (Llama, Mistral, Qwen, etc.)
   ├── Model Path: Load from HuggingFace Hub or local path
   └── Quantization: None, 4-bit, 8-bit options

2. TRAINING METHOD
   ├── SFT (Supervised Fine-Tuning): Standard instruction tuning
   ├── LoRA/QLoRA: Efficient adapter training
   ├── DPO: Preference optimization
   ├── RLHF/PPO: Reinforcement learning from human feedback
   └── Full: Full parameter fine-tuning

3. DATASET
   ├── Built-in datasets: Alpaca, ShareGPT, etc.
   ├── Custom datasets: Upload your own
   └── Dataset preview: See examples before training

4. TRAINING ARGUMENTS
   ├── Epochs, Batch Size, Learning Rate
   ├── LoRA Config (rank, alpha, target modules)
   ├── Optimizer settings
   └── Memory optimization options

5. TRAINING TAB
   ├── Start/Stop training
   ├── Real-time loss curves
   ├── Memory usage monitoring
   └── Checkpoint management

6. CHAT TAB
   ├── Test trained models interactively
   ├── Compare base vs fine-tuned
   └── Adjust generation parameters

7. EXPORT TAB
   ├── Merge LoRA weights
   ├── Export to GGUF (for Ollama)
   └── Push to HuggingFace Hub
"""

print(ui_components)

### Screenshot Walkthrough

The LLaMA Factory UI looks like this:

```
┌─────────────────────────────────────────────────────────────────┐
│  LLaMA Factory                        [Train] [Chat] [Export]   │
├─────────────────────────────────────────────────────────────────┤
│  Model Settings                                                 │
│  ┌─────────────────┐  ┌─────────────────┐                      │
│  │ Model: ▼        │  │ Finetuning: ▼   │                      │
│  │ Llama-3.1-8B    │  │ LoRA            │                      │
│  └─────────────────┘  └─────────────────┘                      │
│                                                                 │
│  Dataset: [alpaca_en ▼]          Quantization: [4-bit ▼]       │
├─────────────────────────────────────────────────────────────────┤
│  Training Arguments                                             │
│  Learning Rate: [2e-4]     Epochs: [3]      Batch Size: [2]    │
│  LoRA Rank: [16]           Alpha: [32]      Dropout: [0.05]    │
├─────────────────────────────────────────────────────────────────┤
│  [▶ Start Training]                         GPU Mem: 45.2 GB   │
│                                                                 │
│  Training Progress ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  85%        │
│  Loss: 0.423 | Step: 850/1000 | ETA: 5:30                      │
│                                                                 │
│  📈 Loss Curve                                                  │
│  1.5 │\                                                        │
│  1.0 │ \___                                                    │
│  0.5 │     \____                                               │
│  0.0 └──────────────                                           │
│      0   250  500  750  1000                                   │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 3: Configuration Walkthrough

Let's go through a complete fine-tuning workflow step by step.

In [ ]:
# Step-by-step configuration for fine-tuning Llama 3.1 8B with LoRA
config_walkthrough = """
FINE-TUNING WALKTHROUGH: Llama 3.1 8B with LoRA
================================================

STEP 1: MODEL CONFIGURATION
---------------------------
1. Open the web UI (http://localhost:7860)
2. In "Model Name", select: Llama-3.1-8B-Instruct
3. Set "Finetuning Method": LoRA
4. Set "Quantization": 4-bit (for memory efficiency)
5. Leave "Adapter Path" empty (we're creating a new adapter)

STEP 2: DATASET CONFIGURATION  
-----------------------------
1. Click "Dataset" dropdown
2. Select a built-in dataset OR:
   - Click "Upload" to add your own
   - Format: JSON or JSONL with "instruction", "input", "output" fields
3. Set "Dataset Split": train (or specify train/val split)
4. Preview data to verify formatting

STEP 3: TRAINING ARGUMENTS
--------------------------
Recommended settings for DGX Spark:

| Setting              | Value     | Notes                          |
|---------------------|-----------|--------------------------------|
| Learning Rate       | 2e-4      | Standard for LoRA              |
| Epochs              | 3         | Adjust based on dataset size   |
| Batch Size          | 4         | Can increase on DGX Spark      |
| Gradient Accum      | 4         | Effective batch = 16           |
| Max Sequence Length | 2048      | Llama 3.1 supports up to 131K  |
| LoRA Rank (r)       | 16        | Balance of efficiency/quality  |
| LoRA Alpha          | 32        | Usually 2x rank                |
| LoRA Dropout        | 0.05      | Light regularization           |
| Target Modules      | All attn  | q,k,v,o projections           |

STEP 4: ADVANCED OPTIONS
------------------------
- Gradient Checkpointing: ON (saves memory)
- BF16: ON (native Blackwell support)
- Flash Attention: ON (faster training)
- DeepSpeed: OFF (single GPU on DGX Spark)

STEP 5: START TRAINING
----------------------
1. Review all settings
2. Click "Start Training"
3. Monitor loss curve and GPU memory
4. Wait for training to complete

STEP 6: TESTING (CHAT TAB)
--------------------------
1. Switch to "Chat" tab
2. Load your trained adapter
3. Test with sample prompts
4. Compare with base model

STEP 7: EXPORT (EXPORT TAB)
---------------------------
1. Switch to "Export" tab
2. Choose export format:
   - Merge LoRA: For standard deployment
   - GGUF: For Ollama/llama.cpp
   - HuggingFace Hub: For sharing
3. Configure export settings
4. Export and verify
"""

print(config_walkthrough)

---

## Part 4: Custom Dataset Upload

Let's prepare a custom dataset for LLaMA Factory.

In [ ]:
import json
from pathlib import Path

# Create a sample dataset in LLaMA Factory format
custom_dataset = [
    {
        "instruction": "Explain the concept of machine learning in simple terms.",
        "input": "",
        "output": "Machine learning is like teaching a computer to learn from examples. Instead of programming every rule, you show the computer many examples and it figures out the patterns. For instance, to recognize cats in photos, you show it thousands of cat pictures until it learns what makes a cat look like a cat."
    },
    {
        "instruction": "What is the difference between AI and machine learning?",
        "input": "",
        "output": "AI (Artificial Intelligence) is the broad goal of making computers think and act intelligently. Machine learning is one way to achieve AI - specifically, by having computers learn from data rather than being explicitly programmed. Think of AI as the destination and machine learning as one popular route to get there."
    },
    {
        "instruction": "Write a Python function based on the following description.",
        "input": "Create a function that takes a list of numbers and returns only the even ones.",
        "output": """def get_even_numbers(numbers):
    \"\"\"Return only the even numbers from a list.\"\"\"
    return [num for num in numbers if num % 2 == 0]

# Example usage:
# get_even_numbers([1, 2, 3, 4, 5, 6])  # Returns [2, 4, 6]"""
    },
    {
        "instruction": "Summarize the following text.",
        "input": "The transformer architecture was introduced in the paper 'Attention is All You Need' in 2017. It uses self-attention mechanisms to process sequences in parallel, which is more efficient than the recurrent approach used by LSTMs and GRUs. Transformers have become the foundation for modern language models like BERT, GPT, and their successors.",
        "output": "Transformers, introduced in 2017, use self-attention for efficient parallel sequence processing, replacing recurrent networks and becoming the foundation for models like BERT and GPT."
    },
    {
        "instruction": "What are the main components of a neural network?",
        "input": "",
        "output": "The main components of a neural network are: 1) Input layer - receives the data, 2) Hidden layers - process the data through weighted connections and activation functions, 3) Output layer - produces the final prediction, 4) Weights and biases - learnable parameters, and 5) Activation functions - introduce non-linearity (like ReLU, sigmoid). These components work together to learn patterns from data."
    }
]

# LLaMA Factory also needs a dataset_info.json file
dataset_info = {
    "custom_data": {
        "file_name": "custom_data.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output"
        }
    }
}

# Save files (in actual use, save to LLaMA Factory's data directory)
print("Sample dataset for LLaMA Factory:")
print(json.dumps(custom_dataset[0], indent=2))
print("\ndataset_info.json entry:")
print(json.dumps(dataset_info, indent=2))

In [ ]:
# Instructions for adding custom dataset to LLaMA Factory
custom_dataset_instructions = """
ADDING CUSTOM DATASET TO LLAMA FACTORY
======================================

METHOD 1: Through Web UI
------------------------
1. Click "Dataset" dropdown
2. Click "Upload Dataset"
3. Upload your JSON/JSONL file
4. Select the dataset in the dropdown

METHOD 2: Manual Addition
-------------------------
1. Place your data file in: LLaMA-Factory/data/
2. Edit data/dataset_info.json to add your dataset:

   {
     "my_dataset": {
       "file_name": "my_data.json",
       "columns": {
         "prompt": "instruction",
         "query": "input",
         "response": "output"
       }
     }
   }

3. Restart the web UI
4. Your dataset appears in the dropdown

SUPPORTED FORMATS
-----------------
1. Alpaca: {"instruction": ..., "input": ..., "output": ...}
2. ShareGPT: {"conversations": [{"from": "human", "value": ...}, ...]}
3. Custom: Specify column mapping in dataset_info.json

TIPS
----
- Keep examples consistent in length and quality
- Aim for 100+ examples minimum
- Include diverse instructions
- Preview data before training
"""

print(custom_dataset_instructions)

---

## Part 5: Command Line Interface

LLaMA Factory also supports command-line training for scripting and automation.

In [ ]:
# CLI training example
cli_training = """
LLAMA FACTORY CLI TRAINING
==========================

Basic training command:
------------------------
llamafactory-cli train \\
    --stage sft \\
    --model_name_or_path meta-llama/Llama-3.1-8B-Instruct \\
    --dataset alpaca_en \\
    --finetuning_type lora \\
    --quantization_bit 4 \\
    --lora_rank 16 \\
    --lora_alpha 32 \\
    --lora_target q_proj,v_proj \\
    --output_dir ./output/llama3-8b-lora \\
    --per_device_train_batch_size 4 \\
    --gradient_accumulation_steps 4 \\
    --lr_scheduler_type cosine \\
    --learning_rate 2e-4 \\
    --num_train_epochs 3 \\
    --bf16 \\
    --logging_steps 10

DPO training:
-------------
llamafactory-cli train \\
    --stage dpo \\
    --model_name_or_path meta-llama/Llama-3.1-8B-Instruct \\
    --adapter_name_or_path ./output/sft-adapter \\
    --dataset dpo_preference_data \\
    --finetuning_type lora \\
    --output_dir ./output/dpo-adapter \\
    --dpo_beta 0.1

Export to GGUF:
---------------
llamafactory-cli export \\
    --model_name_or_path meta-llama/Llama-3.1-8B-Instruct \\
    --adapter_name_or_path ./output/llama3-8b-lora \\
    --export_dir ./output/merged \\
    --export_size 2 \\
    --export_quantization_bit 4 \\
    --export_legacy_format false

Chat with model:
----------------
llamafactory-cli chat \\
    --model_name_or_path meta-llama/Llama-3.1-8B-Instruct \\
    --adapter_name_or_path ./output/llama3-8b-lora \\
    --finetuning_type lora \\
    --quantization_bit 4
"""

print(cli_training)

In [ ]:
# YAML configuration for reproducible training
yaml_config = """
# Save this as: train_config.yaml
# Run with: llamafactory-cli train train_config.yaml

### Model Configuration ###
model_name_or_path: meta-llama/Llama-3.1-8B-Instruct
trust_remote_code: true

### Training Method ###
stage: sft
finetuning_type: lora

### Quantization ###
quantization_bit: 4
quantization_method: bitsandbytes

### Dataset ###
dataset: alpaca_en
template: llama3
cutoff_len: 2048

### LoRA Configuration ###
lora_rank: 16
lora_alpha: 32
lora_dropout: 0.05
lora_target: q_proj,k_proj,v_proj,o_proj

### Training Arguments ###
output_dir: ./output/llama3-8b-sft
per_device_train_batch_size: 4
gradient_accumulation_steps: 4
learning_rate: 2e-4
num_train_epochs: 3
lr_scheduler_type: cosine
warmup_ratio: 0.1

### Memory Optimization ###
bf16: true
gradient_checkpointing: true

### Logging ###
logging_steps: 10
save_steps: 100
save_total_limit: 3
"""

print("YAML Configuration Example:")
print("="*50)
print(yaml_config)

---

## Part 6: Comparison - GUI vs Scripts

Let's compare the two approaches systematically.

In [ ]:
comparison = """
GUI (LLAMA FACTORY) vs SCRIPT-BASED TRAINING
=============================================

LLAMA FACTORY WEB UI
--------------------
Pros:
  ✅ No coding required
  ✅ Visual configuration with validation
  ✅ Real-time training monitoring
  ✅ Built-in chat testing
  ✅ One-click export to multiple formats
  ✅ Presets for common configurations
  ✅ Great for learning and prototyping

Cons:
  ❌ Less flexibility for custom training loops
  ❌ Harder to integrate into CI/CD pipelines
  ❌ Limited access to advanced features
  ❌ Requires running web server
  ❌ Debugging can be harder

SCRIPT-BASED (CUSTOM CODE)
--------------------------
Pros:
  ✅ Maximum control and flexibility
  ✅ Easy to version control
  ✅ Integrates with CI/CD and MLOps
  ✅ Custom training loops possible
  ✅ Advanced debugging and profiling
  ✅ Reproducible experiments

Cons:
  ❌ Requires Python/ML expertise
  ❌ More boilerplate code
  ❌ No visual monitoring (need external tools)
  ❌ Manual configuration management

WHEN TO USE EACH
----------------

Use LLaMA Factory when:
  • Learning fine-tuning for the first time
  • Quick experiments and prototyping
  • You want visual feedback during training
  • Standard fine-tuning (SFT, LoRA, DPO) is sufficient
  • Non-programmers need to run training

Use Scripts when:
  • Production pipelines and automation
  • Custom training logic required
  • Integration with existing MLOps tools
  • Need maximum control over every aspect
  • Running on headless servers (no GUI)
  • Reproducibility is critical

HYBRID APPROACH (RECOMMENDED)
-----------------------------
1. Use LLaMA Factory UI for initial experiments
2. Export the configuration once satisfied
3. Use CLI/scripts for production training
4. Keep LLaMA Factory for quick testing
"""

print(comparison)

---

## Part 7: Troubleshooting Common Issues

In [ ]:
troubleshooting = """
LLAMA FACTORY TROUBLESHOOTING
=============================

ISSUE: Out of Memory (OOM)
--------------------------
Solutions:
1. Reduce batch size to 1
2. Enable gradient checkpointing
3. Use 4-bit quantization
4. Reduce max sequence length
5. Use smaller LoRA rank (8 instead of 16)
6. Clear buffer cache before starting

ISSUE: Web UI Won't Start
-------------------------
Solutions:
1. Check port 7860 is available
2. Try: llamafactory-cli webui --host 0.0.0.0 --port 8080
3. Verify all dependencies installed
4. Check CUDA is accessible

ISSUE: Model Not Showing in Dropdown
------------------------------------
Solutions:
1. Check you have HuggingFace access (for gated models)
2. Set HF_TOKEN environment variable
3. Verify model name is correct
4. Try using local model path

ISSUE: Training Loss Not Decreasing
-----------------------------------
Solutions:
1. Increase learning rate (try 5e-4)
2. Check dataset quality
3. Increase LoRA rank
4. Verify template matches model
5. Train for more epochs

ISSUE: Export/GGUF Conversion Fails
-----------------------------------
Solutions:
1. Install llama-cpp-python
2. Merge LoRA first, then convert
3. Check disk space (70B needs ~140GB)
4. Use correct quantization format

ISSUE: Chat Responses Are Strange
---------------------------------
Solutions:
1. Verify correct chat template
2. Check adapter is properly loaded
3. Try different generation parameters
4. Ensure training completed successfully
"""

print(troubleshooting)

---

## Try It Yourself: Exercises

### Exercise 1: Setup and Explore

1. Install LLaMA Factory
2. Launch the web UI
3. Explore all tabs and options
4. Document 5 features you find interesting

In [ ]:
# Exercise 1: Document your findings
llama_factory_features = [
    # 1. ...
    # 2. ...
    # 3. ...
    # 4. ...
    # 5. ...
]

### Exercise 2: Train with GUI

1. Create a custom dataset (at least 20 examples)
2. Upload it to LLaMA Factory
3. Configure LoRA training
4. Monitor training
5. Test the result in the Chat tab

---

## Checkpoint

You've learned:
- ✅ How to install and launch LLaMA Factory
- ✅ Web UI components and configuration
- ✅ How to add custom datasets
- ✅ CLI and YAML-based training
- ✅ Comparison of GUI vs script approaches

---

## Further Reading

- [LLaMA Factory GitHub](https://github.com/hiyouga/LLaMA-Factory)
- [LLaMA Factory Documentation](https://github.com/hiyouga/LLaMA-Factory/wiki)
- [Supported Models](https://github.com/hiyouga/LLaMA-Factory/blob/main/README.md#supported-models)

---

## Next Steps

**[Task 10.7: Ollama Integration](07-ollama-integration.ipynb)**

Learn to deploy your fine-tuned model to Ollama for easy local inference!

---

## Cleanup

In [ ]:
# Cleanup (if you ran training in this session)
import gc
gc.collect()

# If you have GPU resources to clear:
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU cache cleared")
except ImportError:
    pass

print("Cleanup complete!")